In [1]:
import os
import math
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup
import pickle

In [2]:
df = pd.read_csv("nvdatabase_clean.csv", low_memory=False)
pd.options.display.max_columns = None
df.sample(5)

,user_id,subject,elites,nvresponses,campaigner violence,repressive violence,rating,endyear,method,method_id,theme,classification,country
16285,922,french citizens and soldiers nonviolently defe...,yes,no,no,yes,10.0,1961,121 refusal of public support,121,peace,defense,algeria
977,53,rio de janeiro residents protest world cup and...,no,no,yes,yes,6.0,2016,068 sanctuary,68,human rights,change,brazil
4363,322,guatemalans refuse to serve in civil patrols 1...,yes,no,no,no,8.0,1993,121 refusal of public support,121,peace,change,guatemala
8521,585,thai villagers protest pak mun dam 19912001,no,no,yes,yes,7.0,2001,029 symbolic reclamations,29,democracy,defense,thailand
8773,602,the bolivian cidob launches sixth indigenous m...,no,no,no,yes,6.0,2007,011 records radio and television,11,national-ethnic identity,change,bolivia


In [3]:
users = df[["user_id", "subject", "country", "theme", "classification", "elites", "nvresponses", "campaigner violence", "repressive violence"]]
users = users.drop_duplicates(subset=['user_id'], keep='first')
users = users.rename(columns={'campaigner violence': 'campaigner_violence', 'repressive violence': 'repressive_violence'})
users = users.reset_index(drop=True)

ratings = df[["user_id", "method_id", "rating", "endyear"]]
ratings = ratings.drop_duplicates(keep='first')
ratings = ratings.reset_index(drop=True)

methods = pd.read_csv("nvdatabase_methods.csv", low_memory=False)
methods = methods[["method_id", "method", "genres"]]
#methods['genres'] = methods['genres'].fillna("unknown")
users.head(5)

,user_id,subject,country,theme,classification,elites,nvresponses,campaigner_violence,repressive_violence
0,0,atlanta unions campaign to unionize atlanta ol...,united states,democracy,change,no,no,no,no
1,1,chinese migrant workers protest for equal civi...,china,economic justice,change,no,no,yes,yes
2,2,university of missouri students protest agains...,united states,human rights,change,yes,no,no,yes
3,3,argentinian police force strike for better pay...,argentina,economic justice,change,no,no,no,no
4,4,torres strait soldiers stage stayathome strike...,australia,economic justice,change,no,no,no,no


In [4]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
#users["elites"] = users["elites"].apply(lambda x: f"elites_{x}")
#users["nvresponses"] = users["nvresponses"].apply(lambda x: f"nvresponses_{x}")
#users["campaigner violence"] = users["campaigner violence"].apply(lambda x: f"campaignerviolence_{x}")
#users["repressive violence"] = users["repressive violence"].apply(lambda x: f"repressiveviolence_{x}")
#users["subject"] = users["subject"].apply(lambda x: f"subject_{x}")
#users["clusters"] = users["clusters"].apply(lambda x: f"clusters_{x}")
#users["country"] = users["country"].apply(lambda x: f"country_{x}")
#users["themes"] = users["themes"].apply(lambda x: f"themes_{x}")
#users["classifications"] = users["classifications"].apply(lambda x: f"classifications_{x}")

methods["method_id"] = methods["method_id"].apply(lambda x: f"method_{x}")

ratings["method_id"] = ratings["method_id"].apply(lambda x: f"method_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

ratings.sample(5)

,user_id,method_id,rating,endyear
4484,user_595,method_47,4.0,2010
968,user_129,method_38,6.0,2008
409,user_48,method_26,9.0,2016
4382,user_581,method_47,6.0,2006
9581,user_1163,method_117,10.0,1931


In [5]:
genres = [
    "methods of nonviolent protest and persuasion",
    "formal statements",
    "communications with a wider audience",
    "group representations",
    "symbolic public acts",
    "pressures on individuals",
    "drama and music",
    "processions",
    "honoring the dead",
    "public assemblies",
    "withdrawal and renunciation", 
    "methods of noncooperation",
    "social noncooperation",
    "ostracism of persons",
    "noncooperation with social events, customs and institutions",
    "withdrawal from the social system",
    "economic noncooperation boycotts",
    "action by consumers",
    "action by workers and producers",
    "action by middlemen",
    "action by owners and management",
    "action by holders of financial resources",
    "action by governments",
    "economic noncooperation strikes",
    "symbolic strikes",
    "agricultural strikes",
    "strikes by special groups",
    "ordinary industrial strikes",
    "restricted strikes",
    "multi-industry strikes",
    "combination of strikes and economic closures",
    "political noncooperation",
    "rejection of authority",
    "citizens’ noncooperation with government",
    "citizens’ alternatives to obedience",
    "action by government personnel",
    "domestic governmental action",
    "international governmental action",  
    "methods of nonviolent intervention",
    "psychological intervention",
    "physical intervention",
    "social intervention",
    "economic intervention",
    "political intervention", 
    "additional methods",]

for genre in genres:
    methods[genre] = methods["genres"].apply(lambda values: int(genre in values.split("|")))

methods.head(200)

,method_id,method,genres,methods of nonviolent protest and persuasion,formal statements,communications with a wider audience,group representations,symbolic public acts,pressures on individuals,drama and music,processions,honoring the dead,public assemblies,withdrawal and renunciation,methods of noncooperation,social noncooperation,ostracism of persons,"noncooperation with social events, customs and institutions",withdrawal from the social system,economic noncooperation boycotts,action by consumers,action by workers and producers,action by middlemen,action by owners and management,action by holders of financial resources,action by governments,economic noncooperation strikes,symbolic strikes,agricultural strikes,strikes by special groups,ordinary industrial strikes,restricted strikes,multi-industry strikes,combination of strikes and economic closures,political noncooperation,rejection of authority,citizens’ noncooperation with government,citizens’ alternatives to obedience,action by government personnel,domestic governmental action,international governmental action,methods of nonviolent intervention,psychological intervention,physical intervention,social intervention,economic intervention,political intervention,additional methods
0,method_1,001 public speeches,methods of nonviolent protest and persuasion|f...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,method_2,002 letters of opposition or support,methods of nonviolent protest and persuasion|f...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,method_3,003 declarations by organizations and institut...,methods of nonviolent protest and persuasion|f...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,method_4,004 signed public statements,methods of nonviolent protest and persuasion|f...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,method_5,005 declarations of indictment and intention,methods of nonviolent protest and persuasion|f...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,method_196,196 civil disobedience of neutral laws,methods of nonviolent intervention|political i...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
196,method_197,197 work on without collaboration,methods of nonviolent intervention|political i...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
197,method_198,198 dual sovereignty and parallel government,methods of nonviolent intervention|political i...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
198,method_199,199 nonviolent confinement,additional methods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [6]:
ratings_group = ratings.sort_values(by=["endyear"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "method_ids": list(ratings_group.method_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "endyear": list(ratings_group.endyear.apply(list)),})
ratings_data.head(10)

,user_id,method_ids,ratings,endyear
0,user_0,"[method_4, method_1, method_5, method_16, meth...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[1993, 1993, 1993, 1993, 1993, 1993, 1993, 199..."
1,user_1,"[method_39, method_38, method_1]","[2.0, 2.0, 2.0]","[2011, 2011, 2011]"
2,user_10,"[method_6, method_37, method_34, method_2, met...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[2013, 2013, 2013, 2013, 2013, 2013, 2013, 201..."
3,user_100,"[method_151, method_171, method_47, method_80,...","[4.5, 4.5, 4.5, 4.5, 4.5, 4.5]","[1955, 1955, 1955, 1955, 1955, 1955]"
4,user_1000,"[method_11, method_14, method_15, method_172, ...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[2008, 2008, 2008, 2008, 2008, 2008, 2008, 200..."
5,user_1001,"[method_159, method_15, method_89, method_47]","[7.0, 7.0, 7.0, 7.0]","[2003, 2003, 2003, 2003]"
6,user_1002,"[method_47, method_8, method_7]","[9.0, 9.0, 9.0]","[2010, 2010, 2010]"
7,user_1003,"[method_34, method_124, method_172, method_92,...","[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...","[2010, 2010, 2010, 2010, 2010, 2010, 2010, 201..."
8,user_1004,"[method_106, method_105]","[8.0, 8.0]","[2005, 2005]"
9,user_1005,"[method_159, method_2, method_13, method_34]","[9.0, 9.0, 9.0, 9.0]","[2011, 2011, 2011, 2011]"


In [7]:
sequence_length = 4
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.method_ids = ratings_data.method_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["endyear"]

In [8]:
ratings_data_methods = ratings_data[["user_id", "method_ids"]].explode(
    "method_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_methods, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed.sample(5)

,user_id,method_ids,ratings,subject,country,theme,classification,elites,nvresponses,campaigner_violence,repressive_violence
3546,user_76,"[method_11, method_3, method_7, method_117]","[3.5, 3.5, 3.5, 3.5]",bahrainis protest for democracy februarymarch ...,bahrain,democracy,change,yes,no,yes,yes
3809,user_806,"[method_179, method_124, method_139, method_39]","[6.0, 6.0, 6.0, 6.0]",kosovo albanians resist serbian rule 19901998,kosovo,democracy,change,yes,no,yes,yes
561,user_1144,"[method_60, method_28, method_89, method_126]","[5.0, 5.0, 5.0, 5.0]",haitians demand civilian government and democr...,haiti,democracy,change,yes,no,yes,yes
3625,user_773,"[method_11, method_121, method_104, method_47]","[5.0, 5.0, 5.0, 5.0]",detroit teachers win better working conditions...,united states,economic justice,change,yes,no,no,no
2969,user_664,"[method_3, method_38, method_124, method_34]","[3.0, 3.0, 3.0, 3.0]",kefaya protests mubaraks referendum and reelec...,egypt,democracy,change,yes,no,no,yes


In [9]:
ratings_data_transformed.dropna(subset = ["method_ids"], inplace=True)
ratings_data_transformed = ratings_data_transformed.reset_index(drop=True)
ratings_data_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4463 entries, 0 to 4462
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              4463 non-null   object
 1   method_ids           4463 non-null   object
 2   ratings              4463 non-null   object
 3   subject              4463 non-null   object
 4   country              4463 non-null   object
 5   theme                4463 non-null   object
 6   classification       4463 non-null   object
 7   elites               4463 non-null   object
 8   nvresponses          4463 non-null   object
 9   campaigner_violence  4463 non-null   object
 10  repressive_violence  4463 non-null   object
dtypes: object(11)
memory usage: 383.7+ KB


In [10]:
ratings_data_transformed.method_ids = ratings_data_transformed.method_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

#del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"method_ids": "sequence_method_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

In [11]:
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)

## Define metadata

In [12]:
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
"method_id": list(methods.method_id.unique()),
"user_id": list(users.user_id.unique()),
"subject": list(users.subject.unique()), 
"country": list(users.country.unique()), 
"theme": list(users.theme.unique()), 
"classification": list(users.classification.unique()), 
"elites": list(users.elites.unique()), 
"nvresponses": list(users.nvresponses.unique()), 
"campaigner_violence": list(users.campaigner_violence.unique()), 
"repressive_violence": list(users.repressive_violence.unique()),
}

USER_FEATURES = ["subject", "country", "theme", "classification", "elites", "nvresponses", "campaigner_violence", "repressive_violence"]

METHOD_FEATURES = ["genres"]

## Create `tf.data.Dataset` for training and evaluation

In [13]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        method_ids_string = features["sequence_method_ids"]
        sequence_method_ids = tf.strings.split(method_ids_string, ",").to_tensor()

        # The last method id in the sequence is the target method.
        features["target_method_id"] = sequence_method_ids[:, -1]
        features["sequence_method_ids"] = sequence_method_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset


## Create model inputs

In [14]:

def create_model_inputs():
    return {
        "user_id": layers.Input(name="user_id", shape=(1,), dtype=tf.string),
        "sequence_method_ids": layers.Input(name="sequence_method_ids", shape=(sequence_length - 1,), dtype=tf.string),
        "target_method_id": layers.Input(name="target_method_id", shape=(1,), dtype=tf.string),
        "sequence_ratings": layers.Input(name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32),
        "subject": layers.Input(name="subject", shape=(1,), dtype=tf.string),
        "country": layers.Input(name="country", shape=(1,), dtype=tf.string),
        "theme": layers.Input(name="theme", shape=(1,), dtype=tf.string),
        "classification": layers.Input(name="classification", shape=(1,), dtype=tf.string),
        "elites": layers.Input(name="elites", shape=(1,), dtype=tf.string),
        "nvresponses": layers.Input(name="nvresponses", shape=(1,), dtype=tf.string),
        "campaigner_violence": layers.Input(name="campaigner_violence", shape=(1,), dtype=tf.string),
        "repressive_violence": layers.Input(name="repressive_violence", shape=(1,), dtype=tf.string),
    }

## Encode input features

The `encode_input_features` method works as follows:

1. Each categorical user feature is encoded using `layers.Embedding`, with embedding
dimension equals to the square root of the vocabulary size of the feature.
The embeddings of these features are concatenated to form a single input tensor.

2. Each movie in the movie sequence and the target movie is encoded `layers.Embedding`,
where the dimension size is the square root of the number of movies.

3. A multi-hot genres vector for each movie is concatenated with its embedding vector,
and processed using a non-linear `layers.Dense` to output a vector of the same movie
embedding dimensions.

4. A positional embedding is added to each movie embedding in the sequence, and then
multiplied by its rating from the ratings sequence.

5. The target movie embedding is concatenated to the sequence movie embeddings, producing
a tensor with the shape of `[batch size, sequence length, embedding size]`, as expected
by the attention layer for the transformer architecture.

6. The method returns a tuple of two elements:  `encoded_transformer_features` and
`encoded_other_features`.

In [15]:
def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_method_features=True,
):

    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices.
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions.
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations.
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create a method embedding encoder
    method_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["method_id"]
    method_embedding_dims = int(math.sqrt(len(method_vocabulary)))
    # Create a lookup to convert string values to integer indices.
    method_index_lookup = StringLookup(
        vocabulary=method_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="method_index_lookup",
    )
    # Create an embedding layer with the specified dimensions.
    method_embedding_encoder = layers.Embedding(
        input_dim=len(method_vocabulary),
        output_dim=method_embedding_dims,
        name=f"method_embedding",
    )
    # Create a vector lookup for method genres.
    genre_vectors = methods[genres].to_numpy()
    method_genres_lookup = layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )
    # Create a processing layer for genres.
    method_embedding_processor = layers.Dense(
        units=method_embedding_dims,
        activation="relu",
        name="process_method_embedding_with_genres",
    )

    ## Define a function to encode a given method id.
    def encode_method(method_id):
        # Convert the string input values into integer indices.
        method_idx = method_index_lookup(method_id)
        method_embedding = method_embedding_encoder(method_idx)
        encoded_method = method_embedding
        if include_method_features:
            method_genres_vector = method_genres_lookup(method_idx)
            encoded_method = method_embedding_processor(
                layers.concatenate([method_embedding, method_genres_vector])
            )
        return encoded_method

    ## Encoding target_method_id
    target_method_id = inputs["target_method_id"]
    encoded_target_method = encode_method(target_method_id)

    ## Encoding sequence method_ids.
    sequence_methods_ids = inputs["sequence_method_ids"]
    encoded_sequence_methods = encode_method(sequence_methods_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=method_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the method.
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)
    # Add the positional encoding to the method encodings and multiply them by rating.
    encoded_sequence_methods_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_methods + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs.
    for encoded_method in tf.unstack(
        encoded_sequence_methods_with_poistion_and_rating, axis=1
    ):
        encoded_transformer_features.append(tf.expand_dims(encoded_method, 1))
    encoded_transformer_features.append(encoded_target_method)

    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )

    return encoded_transformer_features, encoded_other_features


## Create a BST model

In [16]:
include_user_id = False
include_user_features = False
include_method_features = False

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_method_features
    )

    # Create a multi-headed attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/numeric.py:2453: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [17]:
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5)

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

Epoch 1/5
15/15 [==============================] - 2s 44ms/step - loss: 34.7540 - mean_absolute_error: 5.6346
Epoch 2/5
15/15 [==============================] - 0s 10ms/step - loss: 14.6824 - mean_absolute_error: 3.6833
Epoch 3/5
15/15 [==============================] - 0s 8ms/step - loss: 7.4524 - mean_absolute_error: 2.5041
Epoch 4/5
15/15 [==============================] - 0s 9ms/step - loss: 4.0903 - mean_absolute_error: 1.7552
Epoch 5/5
15/15 [==============================] - 0s 10ms/step - loss: 2.1177 - mean_absolute_error: 1.2131
Test MAE: 3.033


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_method_ids (InputLaye  [(None, 3)]         0           []                               
 r)                                                                                               
                                                                                                  
 method_index_lookup (StringLoo  multiple            0           ['target_method_id[0][0]',       
 kup)                                                             'sequence_method_ids[0][0]']    
                                                                                                  
 method_embedding (Embedding)   multiple             2800        ['method_index_lookup[0][0]',    
                                                                  'method_index_lookup[1][0]']

 r)                                                                                               
                                                                                                  
 classification (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 country (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 elites (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 nvresponses (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 repressiv

In [22]:
y_pred = model.predict(test_dataset)
print(y_pred[1:10])

[[ 1.2873763 ]
 [ 3.9561436 ]
 [ 6.6657996 ]
 [ 7.4168215 ]
 [-0.26538655]
 [-0.23561409]
 [ 7.3043427 ]
 [ 2.1633368 ]
 [ 0.21790522]]


In [ ]:
print(model.predict([{
    "user_id": input("user_1"),
    "sequence_method_ids": input("[method_39, method_38, method_1]"), 
    "target_method_id": input("method_2"),
    "sequence_ratings": input("[2.0, 2.0, 2.0]"),
    "subject": input("Chinese Migrant Workers Protest for Equal Civil Rights 2011"),
    "country": input("china"),
    "theme": input("economic justice"),
    "classification": input("change"),
    "elites": input("no"),
    "nvresponses": input("no"),
    "campaigner_violence": input("yes"),
    "repressive_violence": input("yes"),

}]))